In [2]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector
from keras.preprocessing.sequence import pad_sequences

import data_cleaning as dc

In [2]:
test_person = pd.read_json(r'week_sleep_data.json', orient='records', lines=True)
st_df = pd.read_json(r"datasets/streamlit_sleep_data.json", orient='records', lines=True)
person_1 = pd.read_json(r'datasets/bangnon_33.json', orient='records', lines=True)
person_2 = pd.read_json(r'datasets/bertablabla.json', lines=True)
person_3 = pd.read_json(r'datasets/boom_90.json', lines=True)
person_4 = pd.read_json(r'datasets/westbrook_30days.json', lines=True)


# test data
syahid_21 = pd.read_json(r'datasets/sleep_data_til_21_syahid.json', orient='records', lines=True)
liza_21 = pd.read_json(r'datasets/sleep_data_til_21_liza.json', orient='records', lines=True)

In [20]:
# assign temporary unique id to each person

dfs = [test_person, person_1, person_2, person_3, person_4]
for num, df in enumerate(dfs):
    df.insert(0, "temp_id", num)

In [4]:
# merge all the sleepDTO df
result_df = pd.DataFrame()
for df in dfs:
    temp_id = df['temp_id']
    daily_df = dc.normalize_and_rename_columns(df, 'dailySleepDTO', 'dailySleepDTO')

    # combine temp_id column to sleepDTO accordingly
    combined_df = pd.concat([temp_id, daily_df], axis=1)

    # append all the results together
    result_df = pd.concat([result_df, combined_df], axis=0)

In [5]:
result_df.head()

,temp_id,id,userProfilePK,calendarDate,sleepTimeSeconds,napTimeSeconds,sleepWindowConfirmed,sleepWindowConfirmationType,sleepStartTimestampGMT,sleepEndTimestampGMT,...,sleepScores.lightPercentage.idealStartInSeconds,sleepScores.lightPercentage.idealEndInSeconds,sleepScores.deepPercentage.value,sleepScores.deepPercentage.qualifierKey,sleepScores.deepPercentage.optimalStart,sleepScores.deepPercentage.optimalEnd,sleepScores.deepPercentage.idealStartInSeconds,sleepScores.deepPercentage.idealEndInSeconds,dailyNapDTOS,userNote
0,0,NaN,116790105,2023-10-03,NaN,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,116790105,2023-10-04,NaN,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,116790105,2023-10-05,NaN,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,116790105,2023-10-06,NaN,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,116790105,2023-10-07,NaN,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data cleaning

In [6]:
# delete untracked nights
result_df = result_df.dropna(subset=["id"])

cols_to_delete = [
    "userProfilePK","napTimeSeconds", "sleepWindowConfirmed", "sleepQualityTypePK", "sleepWindowConfirmationType","sleepStartTimestampGMT",
    "sleepEndTimestampGMT","autoSleepStartTimestampGMT","autoSleepEndTimestampGMT","sleepResultTypePK",
    "unmeasurableSleepSeconds", "deviceRemCapable", "retro", "sleepFromDevice", "sleepScores.totalDuration.optimalStart",
    "sleepScores.totalDuration.optimalEnd", "sleepScores.stress.optimalStart", "sleepScores.stress.optimalEnd",
    "sleepScores.awakeCount.optimalStart","sleepScores.awakeCount.optimalEnd", "sleepScores.overall.qualifierKey",
    "sleepScores.remPercentage.qualifierKey", "sleepScores.remPercentage.optimalStart","sleepScores.remPercentage.optimalEnd",
    "sleepScores.restlessness.optimalStart", "sleepScores.restlessness.optimalEnd", "sleepScores.lightPercentage.qualifierKey",
    "sleepScores.lightPercentage.optimalStart", "sleepScores.lightPercentage.optimalEnd", "sleepScores.deepPercentage.qualifierKey",
    "sleepScores.deepPercentage.optimalStart","sleepScores.deepPercentage.optimalEnd"
    ]

# delete irrelevant columns
result_df  = result_df.drop(cols_to_delete,axis=1)

# change strings to numerical values
for col in result_df.columns:
    if col.endswith("qualifierKey"):
        new_name = col.replace("qualifierKey","value")
        result_df[new_name] = result_df[col].apply(dc.convert_num)
        result_df.drop(col,axis=1,inplace=True)


### Interpolate time series data

In [16]:
test_person = dc.delete_untracked_nights(test_person) # erste Methode
dc.featurevector_timeseries(test_person)

c:\Users\Asus\Desktop\DH_project\Garmin_Watch\data_cleaning.py:106: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


(                                     2023-10-13 00:36:00  2023-10-13 00:37:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBattery                                     0.0                  0.0   
 sleepHeartRate                                       0.0                  0.0   
 wellnessEpochRespirationDataDTOList                  0.0                  0.0   
 sleepLevels                                          1.0                  1.0   
 
                                      2023-10-13 00:38:00  2023-10-13 00:39:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBatt

In [17]:
person_4 = dc.delete_untracked_nights(person_4) # von Streamlit
dc.featurevector_timeseries(person_4)

c:\Users\Asus\Desktop\DH_project\Garmin_Watch\data_cleaning.py:106: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


(                                     2023-11-15 01:18:00  2023-11-15 01:19:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBattery                                     0.0                  0.0   
 sleepHeartRate                                       0.0                  0.0   
 wellnessEpochRespirationDataDTOList                  0.0                  0.0   
 sleepLevels                                          1.0                  1.0   
 
                                      2023-11-15 01:20:00  2023-11-15 01:21:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBatt

In [18]:
st_df = dc.delete_untracked_nights(st_df)
dc.featurevector_timeseries(st_df)

c:\Users\Asus\Desktop\DH_project\Garmin_Watch\data_cleaning.py:106: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


(                                     2023-10-13 00:36:00  2023-10-13 00:37:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBattery                                     0.0                  0.0   
 sleepHeartRate                                       0.0                  0.0   
 wellnessEpochRespirationDataDTOList                  0.0                  0.0   
 sleepLevels                                          1.0                  1.0   
 
                                      2023-10-13 00:38:00  2023-10-13 00:39:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBatt

In [19]:
test_person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        4 non-null      object 
 1   sleepMovement                        4 non-null      object 
 2   remSleepData                         4 non-null      float64
 3   sleepLevels                          4 non-null      object 
 4   sleepRestlessMoments                 4 non-null      object 
 5   restlessMomentsCount                 4 non-null      float64
 6   wellnessEpochRespirationDataDTOList  4 non-null      object 
 7   sleepHeartRate                       4 non-null      object 
 8   sleepStress                          4 non-null      object 
 9   sleepBodyBattery                     4 non-null      object 
 10  hrvData                              4 non-null      object 
 11  avgOvernightHrv                     

In [20]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        4 non-null      object 
 1   sleepMovement                        4 non-null      object 
 2   remSleepData                         4 non-null      float64
 3   sleepLevels                          4 non-null      object 
 4   sleepRestlessMoments                 4 non-null      object 
 5   restlessMomentsCount                 4 non-null      float64
 6   wellnessEpochRespirationDataDTOList  4 non-null      object 
 7   sleepHeartRate                       4 non-null      object 
 8   sleepStress                          4 non-null      object 
 9   sleepBodyBattery                     4 non-null      object 
 10  hrvData                              4 non-null      object 
 11  avgOvernightHrv                     

In [21]:
person_4.info() # use case for drop all NaN will cause huge loss

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        2 non-null      object 
 1   sleepMovement                        2 non-null      object 
 2   remSleepData                         2 non-null      float64
 3   sleepLevels                          2 non-null      object 
 4   sleepRestlessMoments                 2 non-null      object 
 5   restlessMomentsCount                 2 non-null      float64
 6   wellnessEpochRespirationDataDTOList  2 non-null      object 
 7   sleepHeartRate                       2 non-null      object 
 8   sleepStress                          2 non-null      object 
 9   sleepBodyBattery                     2 non-null      object 
 10  hrvData                              2 non-null      object 
 11  avgOvernightHrv                     

In [30]:
syahid_21 = dc.delete_untracked_nights(syahid_21)
dc.featurevector_timeseries(syahid_21)

KeyError: ['restlessMomentsCount']

In [40]:
syahid_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dailySleepDTO  15 non-null     object 
 1   sleepMovement  15 non-null     object 
 2   remSleepData   0 non-null      float64
 3   sleepLevels    15 non-null     object 
dtypes: float64(1), object(3)
memory usage: 608.0+ bytes


In [33]:
liza_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        14 non-null     object 
 1   sleepMovement                        13 non-null     object 
 2   remSleepData                         14 non-null     float64
 3   sleepLevels                          13 non-null     object 
 4   sleepRestlessMoments                 14 non-null     object 
 5   restlessMomentsCount                 14 non-null     float64
 6   wellnessEpochRespirationDataDTOList  13 non-null     object 
 7   sleepHeartRate                       13 non-null     object 
 8   sleepStress                          13 non-null     object 
 9   sleepBodyBattery                     13 non-null     object 
 10  hrvData                              14 non-null     object 
 11  avgOvernightHrv                   

In [31]:
liza_21 = dc.delete_untracked_nights(liza_21)
featurevector_timeseries(liza_21)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6628\2458784053.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


TypeError: 'NoneType' object is not iterable

In [69]:
liza_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        14 non-null     object 
 1   sleepMovement                        13 non-null     object 
 2   remSleepData                         14 non-null     float64
 3   sleepLevels                          13 non-null     object 
 4   sleepRestlessMoments                 14 non-null     object 
 5   restlessMomentsCount                 14 non-null     float64
 6   wellnessEpochRespirationDataDTOList  13 non-null     object 
 7   sleepHeartRate                       13 non-null     object 
 8   sleepStress                          13 non-null     object 
 9   sleepBodyBattery                     13 non-null     object 
 10  hrvData                              14 non-null     object 
 11  avgOvernightHrv                   

In [72]:
liza_21.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        13 non-null     object 
 1   sleepMovement                        13 non-null     object 
 2   remSleepData                         13 non-null     float64
 3   sleepLevels                          13 non-null     object 
 4   sleepRestlessMoments                 13 non-null     object 
 5   restlessMomentsCount                 13 non-null     float64
 6   wellnessEpochRespirationDataDTOList  13 non-null     object 
 7   sleepHeartRate                       13 non-null     object 
 8   sleepStress                          13 non-null     object 
 9   sleepBodyBattery                     13 non-null     object 
 10  hrvData                              13 non-null     object 
 11  avgOvernightHrv                      13

In [120]:
liza_21

,dailySleepDTO,sleepMovement,remSleepData,sleepLevels,sleepRestlessMoments,restlessMomentsCount,wellnessEpochRespirationDataDTOList,sleepHeartRate,sleepStress,sleepBodyBattery,hrvData,avgOvernightHrv,hrvStatus,bodyBatteryChange,restingHeartRate
0,"{'id': 1701986940000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-07T21:09:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-07T22:09:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1701993060000}, {'va...",34.0,"[{'startTimeGMT': 1701986940000, 'respirationV...","[{'value': 71, 'startGMT': 1701986880000}, {'v...","[{'value': 16, 'startGMT': 1701986760000}, {'v...","[{'value': 11, 'startGMT': 1701986760000}, {'v...","[{'value': 66.0, 'startGMT': 1701987153000}, {...",47.0,NONE,58.0,69.0
1,"{'id': 1702078860000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-08T22:41:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-08T23:41:00.0', 'endGMT...","[{'value': 2, 'startGMT': 1702082640000}, {'va...",38.0,"[{'startTimeGMT': 1702078860000, 'respirationV...","[{'value': 82, 'startGMT': 1702078800000}, {'v...","[{'value': 32, 'startGMT': 1702078740000}, {'v...","[{'value': 12, 'startGMT': 1702078740000}, {'v...","[{'value': 35.0, 'startGMT': 1702078952000}, {...",34.0,NONE,26.0,75.0
2,"{'id': 1702171920000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-10T00:32:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-10T01:32:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1702174440000}, {'va...",25.0,"[{'startTimeGMT': 1702171920000, 'respirationV...","[{'value': 76, 'startGMT': 1702171920000}, {'v...","[{'value': 19, 'startGMT': 1702171800000}, {'v...","[{'value': 7, 'startGMT': 1702171800000}, {'va...","[{'value': 49.0, 'startGMT': 1702172166000}, {...",37.0,NONE,33.0,73.0
3,"{'id': 1702254240000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-10T23:24:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-11T00:24:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1702257300000}, {'va...",23.0,"[{'startTimeGMT': 1702254240000, 'respirationV...","[{'value': 84, 'startGMT': 1702254240000}, {'v...","[{'value': 30, 'startGMT': 1702254060000}, {'v...","[{'value': 19, 'startGMT': 1702254060000}, {'v...","[{'value': 31.0, 'startGMT': 1702254249000}, {...",45.0,NONE,37.0,67.0
4,"{'id': 1702340700000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-11T23:25:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-12T00:25:00.0', 'endGMT...","[{'value': 2, 'startGMT': 1702340880000}, {'va...",36.0,"[{'startTimeGMT': 1702340700000, 'respirationV...","[{'value': 76, 'startGMT': 1702340640000}, {'v...","[{'value': 15, 'startGMT': 1702340640000}, {'v...","[{'value': 15, 'startGMT': 1702340640000}, {'v...","[{'value': 67.0, 'startGMT': 1702340978000}, {...",53.0,NONE,66.0,69.0
5,"{'id': 1702425900000, 'userProfilePK': 1167901...",None,1.0,None,"[{'value': 1, 'startGMT': 1702427880000}, {'va...",32.0,None,None,None,None,"[{'value': 68.0, 'startGMT': 1702426178000}, {...",57.0,NONE,NaN,66.0
6,"{'id': 1702510140000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-13T22:29:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-13T23:29:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1702512120000}, {'va...",29.0,"[{'startTimeGMT': 1702510140000, 'respirationV...","[{'value': 73, 'startGMT': 1702510080000}, {'v...","[{'value': 20, 'startGMT': 1702510020000}, {'v...","[{'value': 26, 'startGMT': 1702510020000}, {'v...","[{'value': 53.0, 'startGMT': 1702510345000}, {...",45.0,NONE,41.0,69.0
7,"{'id': 1702596000000, 'userProfilePK': 1167901...","[{'startGMT': '2023-12-14T22:20:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-12-14T23:20:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1702596540000}, {'va...",32.0,"[{'startTimeGMT': 1702596000000, 'respirationV...","[{'value': 74, 'startGMT': 1702596000000}, {'v...","[{'value': 21, 'startGMT': 1702595880000}, {'v...","[{'value': 21, 'startGMT': 1702595880000}, {'v...","[{'value': 55.0, 'startGMT': 1702596145000}, {...",49.0,NONE,60.0,70.0
8,"{'id': 1702680600000, 'userProfilePK': 1167

In [6]:
liza_full = liza_21.dropna().reset_index()
liza_df, _ = featurevector_timeseries(liza_full)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6628\2458784053.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


In [101]:
liza_df

,2023-12-07 22:09:00,2023-12-07 22:10:00,2023-12-07 22:11:00,2023-12-07 22:12:00,2023-12-07 22:13:00,2023-12-07 22:14:00,2023-12-07 22:15:00,2023-12-07 22:16:00,2023-12-07 22:17:00,2023-12-07 22:18:00,...,2023-12-21 08:31:00,2023-12-21 08:32:00,2023-12-21 08:33:00,2023-12-21 08:34:00,2023-12-21 08:35:00,2023-12-21 08:36:00,2023-12-21 08:37:00,2023-12-21 08:38:00,2023-12-21 08:39:00,2023-12-21 08:40:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.0,1.0,1.0,1.0,1.000000,0.000000,0.0,0.0
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51.000000,51.400000,51.8,52.2,52.6,53.0,0.000000,0.000000,0.0,0.0
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.333333,23.666667,24.0,23.0,22.0,21.0,22.333333,23.666667,25.0,0.0
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51.333333,51.666667,52.0,52.0,52.0,52.0,52.000000,52.000000,52.0,0.0
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,78.500000,77.000000,75.5,74.0,75.0,76.0,74.500000,73.000000,71.5,70.0
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.500000,15.000000,15.5,16.0,15.5,15.0,14.500000,14.000000,14.0,14.0
sleepLevels,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [32]:
date_index = (liza_df.columns)
date_index.date
np.unique(date_index.date) # 12/12 is still there (interpolated values)

array([datetime.date(2023, 12, 7), datetime.date(2023, 12, 8),
       datetime.date(2023, 12, 9), datetime.date(2023, 12, 10),
       datetime.date(2023, 12, 11), datetime.date(2023, 12, 12),
       datetime.date(2023, 12, 13), datetime.date(2023, 12, 14),
       datetime.date(2023, 12, 15), datetime.date(2023, 12, 16),
       datetime.date(2023, 12, 17), datetime.date(2023, 12, 18),
       datetime.date(2023, 12, 19), datetime.date(2023, 12, 20),
       datetime.date(2023, 12, 21)], dtype=object)

In [11]:
selected_date = pd.Timestamp('2023-12-12')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-12 00:25:00,2023-12-12 00:26:00,2023-12-12 00:27:00,2023-12-12 00:28:00,2023-12-12 00:29:00,2023-12-12 00:30:00,2023-12-12 00:31:00,2023-12-12 00:32:00,2023-12-12 00:33:00,2023-12-12 00:34:00,...,2023-12-12 09:54:00,2023-12-12 09:55:00,2023-12-12 09:56:00,2023-12-12 09:57:00,2023-12-12 09:58:00,2023-12-12 09:59:00,2023-12-12 10:00:00,2023-12-12 10:01:00,2023-12-12 10:02:00,2023-12-12 10:03:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.666667,2.0,1.666667,1.333333,1.000000,1.000000,1.0,1.0,0.0,0.0
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.000000,47.4,46.800000,46.200000,45.600000,45.000000,0.0,0.0,0.0,0.0
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.000000,26.0,30.000000,34.000000,31.333333,28.666667,26.0,25.0,24.0,23.0
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,83.000000,83.0,83.000000,83.000000,82.333333,81.666667,81.0,81.0,81.0,81.0
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,76.000000,77.0,78.000000,79.000000,80.000000,80.000000,80.0,79.5,79.0,0.0
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.000000,13.0,13.000000,13.000000,13.000000,13.500000,14.0,14.0,14.0,0.0
sleepLevels,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [12]:
selected_date = pd.Timestamp('2023-12-11')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-11 00:24:00,2023-12-11 00:25:00,2023-12-11 00:26:00,2023-12-11 00:27:00,2023-12-11 00:28:00,2023-12-11 00:29:00,2023-12-11 00:30:00,2023-12-11 00:31:00,2023-12-11 00:32:00,2023-12-11 00:33:00,...,2023-12-11 07:12:00,2023-12-11 07:13:00,2023-12-11 07:14:00,2023-12-11 07:15:00,2023-12-11 07:16:00,2023-12-11 07:17:00,2023-12-11 07:18:00,2023-12-11 07:19:00,2023-12-11 07:20:00,2023-12-11 07:21:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.000000,1.000000,1.0,1.333333,1.666667,2.0,0.000000,0.000000,0.0
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.6,44.800000,43.000000,45.8,48.600000,51.400000,54.2,57.000000,0.000000,0.0
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,16.333333,15.666667,15.0,17.666667,20.333333,23.0,24.333333,25.666667,27.0
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,55.0,55.333333,55.666667,56.0,56.000000,56.000000,56.0,56.000000,56.000000,56.0
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,77.0,74.500000,72.000000,72.5,73.000000,73.500000,74.0,77.500000,81.000000,0.0
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,14.000000,14.000000,14.0,14.000000,13.500000,13.0,12.500000,12.000000,0.0
sleepLevels,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [13]:
selected_date = pd.Timestamp('2023-12-13')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-13 23:29:00,2023-12-13 23:30:00,2023-12-13 23:31:00,2023-12-13 23:32:00,2023-12-13 23:33:00,2023-12-13 23:34:00,2023-12-13 23:35:00,2023-12-13 23:36:00,2023-12-13 23:37:00,2023-12-13 23:38:00,...,2023-12-13 23:50:00,2023-12-13 23:51:00,2023-12-13 23:52:00,2023-12-13 23:53:00,2023-12-13 23:54:00,2023-12-13 23:55:00,2023-12-13 23:56:00,2023-12-13 23:57:00,2023-12-13 23:58:00,2023-12-13 23:59:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepLevels,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
selected_date = pd.Timestamp('2023-12-14')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-14 00:00:00,2023-12-14 00:01:00,2023-12-14 00:02:00,2023-12-14 00:03:00,2023-12-14 00:04:00,2023-12-14 00:05:00,2023-12-14 00:06:00,2023-12-14 00:07:00,2023-12-14 00:08:00,2023-12-14 00:09:00,...,2023-12-14 23:50:00,2023-12-14 23:51:00,2023-12-14 23:52:00,2023-12-14 23:53:00,2023-12-14 23:54:00,2023-12-14 23:55:00,2023-12-14 23:56:00,2023-12-14 23:57:00,2023-12-14 23:58:00,2023-12-14 23:59:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sleepLevels,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
selected_date = pd.Timestamp('2023-12-15')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-15 00:00:00,2023-12-15 00:01:00,2023-12-15 00:02:00,2023-12-15 00:03:00,2023-12-15 00:04:00,2023-12-15 00:05:00,2023-12-15 00:06:00,2023-12-15 00:07:00,2023-12-15 00:08:00,2023-12-15 00:09:00,...,2023-12-15 23:50:00,2023-12-15 23:51:00,2023-12-15 23:52:00,2023-12-15 23:53:00,2023-12-15 23:54:00,2023-12-15 23:55:00,2023-12-15 23:56:00,2023-12-15 23:57:00,2023-12-15 23:58:00,2023-12-15 23:59:00
sleepRestlessMoments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
hrvData,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,35.00,37.800000,40.600000,43.4,46.2,49.0,49.6,50.200000,50.800000
sleepStress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,22.00,20.666667,19.333333,18.0,18.0,18.0,18.0,18.666667,19.333333
sleepBodyBattery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,26.00,26.000000,26.000000,26.0,26.0,26.0,26.0,26.000000,26.000000
sleepHeartRate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,76.0,76.00,76.000000,75.000000,74.0,73.5,73.0,72.5,72.000000,72.000000
wellnessEpochRespirationDataDTOList,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.5,14.75,15.000000,14.500000,14.0,13.5,13.0,13.5,14.000000,14.000000
sleepLevels,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.00,3.000000,3.000000,3.0,3.0,3.0,3.0,3.000000,3.000000


In [119]:
selected_date = pd.Timestamp('2023-12-16')

# Filter columns based on the date
columns_for_selected_date = liza_df.columns[liza_df.columns.date == selected_date.date()]

# Slice the DataFrame to include only columns for the selected date
liza_df_selected_date = liza_df[columns_for_selected_date]
liza_df_selected_date

,2023-12-16 00:00:00,2023-12-16 00:01:00,2023-12-16 00:02:00,2023-12-16 00:03:00,2023-12-16 00:04:00,2023-12-16 00:05:00,2023-12-16 00:06:00,2023-12-16 00:07:00,2023-12-16 00:08:00,2023-12-16 00:09:00,...,2023-12-16 10:45:00,2023-12-16 10:46:00,2023-12-16 10:47:00,2023-12-16 10:48:00,2023-12-16 10:49:00,2023-12-16 10:50:00,2023-12-16 10:51:00,2023-12-16 10:52:00,2023-12-16 10:53:00,2023-12-16 10:54:00
sleepRestlessMoments,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,1.0,...,1.333333,1.666667,2.000000,1.833333,1.666667,1.500000,1.333333,1.166667,1.000000,0.0
hrvData,51.4,52.000000,50.600000,49.2,47.800000,46.400000,45.0,45.200000,45.400000,45.6,...,43.200000,42.000000,41.400000,40.800000,40.200000,39.600000,39.000000,0.000000,0.000000,0.0
sleepStress,20.0,20.666667,21.333333,22.0,20.666667,19.333333,18.0,20.333333,22.666667,25.0,...,43.000000,40.666667,38.333333,36.000000,31.666667,27.333333,23.000000,20.666667,18.333333,16.0
sleepBodyBattery,26.0,26.000000,26.000000,26.0,26.000000,26.000000,26.0,26.000000,26.000000,26.0,...,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.0
sleepHeartRate,72.0,72.500000,73.000000,73.0,73.000000,73.000000,73.0,73.000000,73.000000,73.5,...,79.500000,82.000000,79.500000,77.000000,78.500000,80.000000,79.000000,78.000000,76.500000,75.0
wellnessEpochRespirationDataDTOList,14.0,13.500000,13.000000,13.5,14.000000,14.000000,14.0,14.000000,14.000000,13.5,...,13.500000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,12.500000,12.0
sleepLevels,3.0,3.000000,3.000000,3.0,3.000000,3.000000,3.0,3.000000,3.000000,3.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


### drop na from p1

In [79]:
p1_ = person_1.dropna().reset_index()
featurevector_timeseries(p1_)

C:\Users\Asus\AppData\Local\Temp\ipykernel_1476\3410806315.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


(                                     2023-12-05 02:53:00  2023-12-05 02:54:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBattery                                     0.0                  0.0   
 sleepHeartRate                                       0.0                  0.0   
 wellnessEpochRespirationDataDTOList                  0.0                  0.0   
 sleepLevels                                          1.0                  1.0   
 
                                      2023-12-05 02:55:00  2023-12-05 02:56:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBatt

In [81]:
p1_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   index                                10 non-null     int64  
 1   dailySleepDTO                        10 non-null     object 
 2   sleepMovement                        10 non-null     object 
 3   remSleepData                         10 non-null     float64
 4   sleepLevels                          10 non-null     object 
 5   sleepRestlessMoments                 10 non-null     object 
 6   restlessMomentsCount                 10 non-null     float64
 7   wellnessEpochRespirationDataDTOList  10 non-null     object 
 8   sleepHeartRate                       10 non-null     object 
 9   sleepStress                          10 non-null     object 
 10  sleepBodyBattery                     10 non-null     object 
 11  restingHeartRate                   

In [83]:
person_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        34 non-null     object 
 1   sleepMovement                        32 non-null     object 
 2   remSleepData                         33 non-null     float64
 3   sleepLevels                          32 non-null     object 
 4   sleepRestlessMoments                 33 non-null     object 
 5   restlessMomentsCount                 33 non-null     float64
 6   wellnessEpochRespirationDataDTOList  29 non-null     object 
 7   sleepHeartRate                       29 non-null     object 
 8   sleepStress                          29 non-null     object 
 9   sleepBodyBattery                     29 non-null     object 
 10  restingHeartRate                     33 non-null     float64
 11  hrvData                           

In [46]:
person_1.iloc[:20].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        20 non-null     object 
 1   sleepMovement                        20 non-null     object 
 2   remSleepData                         20 non-null     float64
 3   sleepLevels                          20 non-null     object 
 4   sleepRestlessMoments                 20 non-null     object 
 5   restlessMomentsCount                 20 non-null     float64
 6   wellnessEpochRespirationDataDTOList  19 non-null     object 
 7   sleepHeartRate                       19 non-null     object 
 8   sleepStress                          19 non-null     object 
 9   sleepBodyBattery                     19 non-null     object 
 10  restingHeartRate                     20 non-null     float64
 11  hrvData                           

### manual NaN removal

In [63]:
p1 = dc.delete_untracked_nights(person_1.iloc[-3:-2]) # manually slice the dataframe without NaN values in SleepMovement
featurevector_timeseries(p1)

C:\Users\Asus\AppData\Local\Temp\ipykernel_1476\3410806315.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  preliminary_featurevector_allnights = preliminary_featurevector_allnights.merge(preliminary_featurevector, left_index=True, right_index=True)


(                                     2023-12-13 02:22:00  2023-12-13 02:23:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBattery                                     0.0                  0.0   
 sleepHeartRate                                       0.0                  0.0   
 wellnessEpochRespirationDataDTOList                  0.0                  0.0   
 sleepLevels                                          1.0                  1.0   
 
                                      2023-12-13 02:24:00  2023-12-13 02:25:00  \
 sleepRestlessMoments                                 0.0                  0.0   
 hrvData                                              0.0                  0.0   
 sleepStress                                          0.0                  0.0   
 sleepBodyBatt

In [64]:
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   dailySleepDTO                        1 non-null      object 
 1   sleepMovement                        1 non-null      object 
 2   remSleepData                         1 non-null      float64
 3   sleepLevels                          1 non-null      object 
 4   sleepRestlessMoments                 1 non-null      object 
 5   restlessMomentsCount                 1 non-null      float64
 6   wellnessEpochRespirationDataDTOList  1 non-null      object 
 7   sleepHeartRate                       1 non-null      object 
 8   sleepStress                          1 non-null      object 
 9   sleepBodyBattery                     1 non-null      object 
 10  restingHeartRate                     1 non-null      float64
 11  hrvData                             

In [65]:
person_1

,dailySleepDTO,sleepMovement,remSleepData,sleepLevels,sleepRestlessMoments,restlessMomentsCount,wellnessEpochRespirationDataDTOList,sleepHeartRate,sleepStress,sleepBodyBattery,restingHeartRate,hrvData,avgOvernightHrv,hrvStatus,bodyBatteryChange
0,"{'id': 1699760400000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-12T02:40:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-12T03:40:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1699762740000}, {'va...",36.0,"[{'startTimeGMT': 1699760400000, 'respirationV...","[{'value': 54, 'startGMT': 1699760400000}, {'v...","[{'value': 10, 'startGMT': 1699760340000}, {'v...","[{'value': 6, 'startGMT': 1699760340000}, {'va...",52.0,None,NaN,None,NaN
1,"{'id': 1699835280000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-12T23:28:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-13T00:28:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1699835640000}, {'va...",43.0,"[{'startTimeGMT': 1699835280000, 'respirationV...","[{'value': 61, 'startGMT': 1699835280000}, {'v...","[{'value': 20, 'startGMT': 1699835220000}, {'v...","[{'value': 13, 'startGMT': 1699835220000}, {'v...",49.0,None,NaN,None,NaN
2,"{'id': 1699931580000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-14T02:13:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-14T03:13:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1699932120000}, {'va...",32.0,"[{'startTimeGMT': 1699931580000, 'respirationV...","[{'value': 64, 'startGMT': 1699931520000}, {'v...","[{'value': 17, 'startGMT': 1699931520000}, {'v...","[{'value': 16, 'startGMT': 1699931520000}, {'v...",52.0,None,NaN,None,NaN
3,"{'id': 1700015880000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-15T01:38:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-15T02:38:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1700016420000}, {'va...",66.0,"[{'startTimeGMT': 1700015880000, 'respirationV...","[{'value': 54, 'startGMT': 1700015880000}, {'v...","[{'value': -1, 'startGMT': 1700015760000}, {'v...","[{'value': 20, 'startGMT': 1700015760000}, {'v...",50.0,None,NaN,None,NaN
4,"{'id': 1700104200000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-16T02:10:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-16T03:10:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1700104560000}, {'va...",25.0,"[{'startTimeGMT': 1700104200000, 'respirationV...","[{'value': 58, 'startGMT': 1700104200000}, {'v...","[{'value': 20, 'startGMT': 1700104140000}, {'v...","[{'value': 17, 'startGMT': 1700104140000}, {'v...",53.0,None,NaN,None,NaN
5,"{'id': 1700193840000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-17T03:04:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-17T04:04:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1700193840000}, {'va...",56.0,"[{'startTimeGMT': 1700193840000, 'respirationV...","[{'value': 49, 'startGMT': 1700193840000}, {'v...","[{'value': 6, 'startGMT': 1700193780000}, {'va...","[{'value': 24, 'startGMT': 1700193780000}, {'v...",48.0,None,NaN,None,NaN
6,"{'id': 1700272260000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-18T00:51:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-18T01:51:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1700275860000}, {'va...",50.0,"[{'startTimeGMT': 1700272260000, 'respirationV...","[{'value': 58, 'startGMT': 1700272200000}, {'v...","[{'value': 19, 'startGMT': 1700272080000}, {'v...","[{'value': 14, 'startGMT': 1700272080000}, {'v...",53.0,None,NaN,None,NaN
7,"{'id': 1700363220000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-19T02:07:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-19T03:07:00.0', 'endGMT...","[{'value': 1, 'startGMT': 1700363940000}, {'va...",48.0,"[{'startTimeGMT': 1700363220000, 'respirationV...","[{'value': 53, 'startGMT': 1700363160000}, {'v...","[{'value': 11, 'startGMT': 1700363160000}, {'v...","[{'value': 25, 'startGMT': 1700363160000}, {'v...",48.0,None,NaN,None,NaN
8,"{'id': 1700448060000, 'userProfilePK': 1167901...","[{'startGMT': '2023-11-20T01:41:00.0', 'endGMT...",1.0,"[{'startGMT': '2023-11-20T02:41:00.0', 'endG